In [0]:
import numpy as np
import scipy as sc
import gensim as g
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import csv

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config IPCompleter.greedy=True
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras import layers
from keras.layers import Flatten

In [8]:
from google.colab import files
uploaded = files.upload()

from google.colab import files
uploaded = files.upload()



Saving test_neg.csv to test_neg (2).csv
Saving train_neg.csv to train_neg (1).csv
Saving val_neg.csv to val_neg (1).csv


In [25]:
from google.colab import files
uploaded = files.upload()


Saving word2vec.bin to word2vec (1).bin


# **DATA PREPROCESSING: LOADING**

In [0]:
files=['train_pos.csv','train_neg.csv','val_pos.csv','val_neg.csv',
       'test_pos.csv','test_neg.csv']

def load_data(file):
    def retreive_from_csv(words):
        ls = [i for i in words]
        return ls
    
    f = open(file)
    lists=[]
 
    for j in f:
        lists.append((retreive_from_csv(eval(j))))
     
    return lists

In [0]:
train_pos_list = load_data(files[0])
train_neg_list=load_data(files[1])

val_pos_list = load_data(files[2])
val_neg_list=load_data(files[3])

test_pos_list=load_data(files[4])
test_neg_list=load_data(files[5])

# DATA PREPROCESSING: GETTING WORD EMBEDDINGS

Word 2 Vec is only on training set and NOT on the entire corpus

In [107]:
from gensim.models import Word2Vec

w2v_saved_model = g.models.KeyedVectors.load_word2vec_format("word2vec.bin",binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [157]:
print('Number of words in this pre-trained w2v model:', len(w2v_saved_model.wv.vocab))
print('Dimension of w2v:', w2v_saved_model.wv.vector_size)

Number of words in this pre-trained w2v model: 128541
Dimension of w2v: 300


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


# DATA PREPROCESSING: ASSIGNING OUTCOME TO CORPUS AS POSITIVE/NEGATIVE

In [0]:
positive_train=pd.DataFrame(columns=['sentence', 'outcome'])
positive_train['sentence'] = train_pos_list
positive_train['outcome']  = [np.array([1,0])]*len(train_pos_list) 

positive_test=pd.DataFrame(columns=['sentence', 'outcome'])
positive_test['sentence'] = test_pos_list
positive_test['outcome']= [np.array([1,0])]*len(test_pos_list)  

positive_val=pd.DataFrame(columns=['sentence', 'outcome'])
positive_val['sentence'] = val_pos_list
positive_val['outcome']= [np.array([1,0])]*len(val_pos_list)    

negitive_train=pd.DataFrame(columns=['sentence', 'outcome'])
negitive_train['sentence'] = train_neg_list
negitive_train['outcome']  = [np.array([0,1])]*len(train_neg_list) 

negitive_test=pd.DataFrame(columns=['sentence', 'outcome'])
negitive_test['sentence'] = test_neg_list
negitive_test['outcome'] = [np.array([0,1])]*len(test_neg_list)  

negitive_val=pd.DataFrame(columns=['sentence', 'outcome'])
negitive_val['sentence'] = val_neg_list
negitive_val['outcome']= [np.array([0,1])]*len(val_neg_list)    

In [0]:
train_df=positive_train.append(negitive_train)
test_df=positive_test.append(negitive_test)
val_df=positive_val.append(negitive_val)

In [0]:
train_df=train_df.sample(frac=1,random_state=10)
train_df.reset_index(inplace=True,drop=True)

test_df=test_df.sample(frac=1,random_state=10)
test_df.reset_index(inplace=True,drop=True)


val_df=val_df.sample(frac=1,random_state=10)
val_df.reset_index(inplace=True,drop=True)

In [144]:
test_df.head()

,sentence,outcome
0,"[detracting, from, any, satisfaction, you, mig...","[0, 1]"
1,"[i, thought, i, was, going, to, get, some, che...","[1, 0]"
2,"[i, 've, played, games, released, in, 2001, th...","[0, 1]"
3,"[this, comes, in, handy, when, you, are, worki...","[1, 0]"
4,"[yet, it, tries, to, be, both, and, does, so, ...","[0, 1]"


# TOKENIZATION OF CORPUS: CONVERTING INTO NUMBERS & PADDING

In [145]:
  tokenizer = Tokenizer(num_words=len(w2v_saved_model.wv.vocab))
  tokenizer.fit_on_texts(np.array(train_df["sentence"]))
  seq = tokenizer.texts_to_sequences([" ".join(sent) for sent in train_df["sentence"]])
  #print('90th Percentile Sentence Length:', np.percentile([len(s) for s in seq], 99))
  X_train = pad_sequences(seq, maxlen=30, padding='post', truncating='post')
  y_train = train_df['outcome']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [0]:
def tokenizing_corpus(lists,token):
  # Convert the sequence of words to sequnce of indices
  seq = token.texts_to_sequences([" ".join(sent) for sent in lists["sentence"]])
  #print('90th Percentile Sentence Length:', np.percentile([len(s) for s in seq], 99))
  X = pad_sequences(seq, maxlen=30, padding='post', truncating='post')
  y = lists['outcome']
  return X,y

In [0]:
X_test,y_test=tokenizing_corpus(test_df,(tokenizer))
X_val,y_val=tokenizing_corpus(val_df,(tokenizer))

In [0]:
y_train = np.array([i for i in y_train])
y_test = np.array([i for i in y_test])
y_val = np.array([i for i in y_val])


In [149]:
y_train.shape

(640000, 2)

# PREPARING THE WEIGHTS

In [0]:
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, 300)) # +1 is because the matrix indices start with 0

for word, i in tokenizer.word_index.items(): # i=0 is the embedding for the zero padding
    try:
        embeddings_vector = w2v_saved_model[word]
    except KeyError:
        embeddings_vector = None
    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector


In [138]:
embeddings_matrix.shape


(128680, 300)

# BASELINE MODEL DEVELOPMENT

In [165]:
model = Sequential()
model.add(Embedding(input_dim=(len(tokenizer.word_index)+1),output_dim=300, input_length=30, weights = [embeddings_matrix], trainable=False, name='word_embedding_layer', mask_zero=False))


model.add(Flatten())

model.add(Dropout(rate=0.2, name='dropout_layer'))

model.add(layers.Dense(32,  name='hidden_layer',activation = "relu", kernel_regularizer=l2(0.01)))

model.add(layers.Dense(2, activation='softmax', name='output_layer'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 30, 300)           38604000  
_________________________________________________________________
flatten_28 (Flatten)         (None, 9000)              0         
_________________________________________________________________
dropout_layer (Dropout)      (None, 9000)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 32)                288032    
_________________________________________________________________
output_layer (Dense)         (None, 2)                 66        
Total params: 38,892,098
Trainable params: 288,098
Non-trainable params: 38,604,000
_________________________________________________________________


# TRAINING THE BASELINE MODEL

In [167]:
model.fit(X_train, y_train, batch_size=256, epochs=5,validation_data=(X_val, y_val))

Train on 640000 samples, validate on 80000 samples
Epoch 1/5
640000/640000 [==============================] - 171s 268us/step - loss: 0.5837 - acc: 0.7170 - val_loss: 0.6136 - val_acc: 0.6853
Epoch 2/5
640000/640000 [==============================] - 171s 267us/step - loss: 0.5802 - acc: 0.7172 - val_loss: 0.6107 - val_acc: 0.6905
Epoch 3/5
640000/640000 [==============================] - 171s 267us/step - loss: 0.5786 - acc: 0.7189 - val_loss: 0.6126 - val_acc: 0.6835
Epoch 4/5
640000/640000 [==============================] - 171s 268us/step - loss: 0.5787 - acc: 0.7180 - val_loss: 0.6219 - val_acc: 0.6757
Epoch 5/5
640000/640000 [==============================] - 171s 268us/step - loss: 0.5772 - acc: 0.7193 - val_loss: 0.6908 - val_acc: 0.6050


# BASELINE MODEL ACCURACY

In [168]:
score, acc = model.evaluate(X_test, y_test, batch_size=32)


print("Accuracy on Test Set = {0:4.3f}".format(acc))

80000/80000 [==============================] - 19s 232us/step
Accuracy on Test Set = 0.614


# PERFORMANCE TUNING ON VALIDATION SET: TUNING ACTIVATIONS


In [169]:
activations = ["sigmoid", "tanh"]
results=[]
concat_result=[]

for i in range(len(activations)):
        
        # create model
        model = Sequential()
        model.add(Embedding(input_dim=(len(tokenizer.word_index)+1),output_dim=300,input_length=30,weights = [embeddings_matrix], trainable=False, name='word_embedding_layer',mask_zero=False))


        model.add(Flatten())

       

        model.add(layers.Dense(32,  name='hidden_layer',activation = activations[i], kernel_regularizer=l2(0.01)))
        
        model.add(Dropout(rate=0.2, name='dropout_layer'))

        model.add(layers.Dense(2, activation='softmax', name='output_layer'))

        model.summary()

        
        
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

        model.fit(X_train, y_train, batch_size=256, epochs=5,validation_data=(X_val, y_val))

        score, acc = model.evaluate(X_test, y_test, batch_size=32)
        print("Accuracy on Test Set = {0:4.3f}".format(acc))
        
        # Create a column from the list
        
        
        results.append(acc) 
        
   
        concat_result=pd.DataFrame(results)
        results=[]
    
        concat_result
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 30, 300)           38604000  
_________________________________________________________________
flatten_29 (Flatten)         (None, 9000)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 32)                288032    
_________________________________________________________________
dropout_layer (Dropout)      (None, 32)                0         
_________________________________________________________________
output_layer (Dense)         (None, 2)                 66        
Total params: 38,892,098
Trainable params: 288,098
Non-trainable params: 38,604,000
_________________________________________________________________
Train on 640000 samples, validate on 80000 samples
Epoch 1/5
640000/640000 [==============================] - 57s 89us/step 

80000/80000 [==============================] - 6s 76us/step
Accuracy on Test Set = 0.663


,0
0,0.662825


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 30, 300)           38604000  
_________________________________________________________________
flatten_30 (Flatten)         (None, 9000)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 32)                288032    
_________________________________________________________________
dropout_layer (Dropout)      (None, 32)                0         
_________________________________________________________________
output_layer (Dense)         (None, 2)                 66        
Total params: 38,892,098
Trainable params: 288,098
Non-trainable params: 38,604,000
_________________________________________________________________
Train on 640000 samples, validate on 80000 samples
Epoch 1/5
640000/640000 [==============================] - 57s 90us/step 

80000/80000 [==============================] - 6s 78us/step
Accuracy on Test Set = 0.684


,0
0,0.684163


# PERFORMANCE TUNING ON VALIDATION SET: DIFFERENT DROPOUT RATES


In [170]:

dropouts= [0.2,0.25, 0.3, 0.4,0.5]
results_dropout=[]
concat_result_dropout=[]


for i in range(len(dropouts)):
        
        # create model
        model = Sequential()
        model.add(Embedding(input_dim=(len(tokenizer.word_index)+1),output_dim=300,input_length=30,weights = [embeddings_matrix], trainable=False, name='word_embedding_layer',mask_zero=False))


        model.add(Flatten())

        model.add(layers.Dense(32,  name='hidden_layer',activation = 'tanh', kernel_regularizer=l2(0.01)))
        
        model.add(Dropout(rate=dropouts[i], name='dropout_layer'))

        model.add(layers.Dense(2, activation='softmax', name='output_layer'))

        model.summary()

        
        
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

        model.fit(X_train, y_train, batch_size=256, epochs=5,validation_data=(X_val, y_val))

        score, acc = model.evaluate(X_test, y_test, batch_size=32)
        print("Accuracy on Test Set = {0:4.3f}".format(acc))
        
        # Create a column from the list
        
        
        results.append(acc) 
        
   
        concat_result=pd.DataFrame(results)
        results=[]
    
        concat_result    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 30, 300)           38604000  
_________________________________________________________________
flatten_31 (Flatten)         (None, 9000)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 32)                288032    
_________________________________________________________________
dropout_layer (Dropout)      (None, 32)                0         
_________________________________________________________________
output_layer (Dense)         (None, 2)                 66        
Total params: 38,892,098
Trainable params: 288,098
Non-trainable params: 38,604,000
_________________________________________________________________
Train on 640000 samples, validate on 80000 samples
Epoch 1/5
640000/640000 [==============================] - 56s 88us/step 

80000/80000 [==============================] - 6s 77us/step
Accuracy on Test Set = 0.692


,0
0,0.692137


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 30, 300)           38604000  
_________________________________________________________________
flatten_32 (Flatten)         (None, 9000)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 32)                288032    
_________________________________________________________________
dropout_layer (Dropout)      (None, 32)                0         
_________________________________________________________________
output_layer (Dense)         (None, 2)                 66        
Total params: 38,892,098
Trainable params: 288,098
Non-trainable params: 38,604,000
_________________________________________________________________
Train on 640000 samples, validate on 80000 samples
Epoch 1/5
640000/640000 [==============================] - 57s 89us/step 

80000/80000 [==============================] - 6s 79us/step
Accuracy on Test Set = 0.708


...

80000/80000 [==============================] - 6s 80us/step
Accuracy on Test Set = 0.679


,0
0,0.679375


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 30, 300)           38604000  
_________________________________________________________________
flatten_35 (Flatten)         (None, 9000)              0         
_________________________________________________________________
hidden_layer (Dense)         (None, 32)                288032    
_________________________________________________________________
dropout_layer (Dropout)      (None, 32)                0         
_________________________________________________________________
output_layer (Dense)         (None, 2)                 66        
Total params: 38,892,098
Trainable params: 288,098
Non-trainable params: 38,604,000
_________________________________________________________________
Train on 640000 samples, validate on 80000 samples
Epoch 1/5
640000/640000 [==============================] - 58s 91us/step 

80000/80000 [==============================] - 6s 81us/step
Accuracy on Test Set = 0.693


,0
0,0.69305
